# LAB 02: ELT Ingestion & Transformations

**Duration:** ~40 min  
**Day:** 1  
**After module:** M02: ELT Data Ingestion  
**Difficulty:** Beginner-Intermediate

---

## Scenario

> *"Your workspace is ready. Now it's time to build the first data pipeline — load raw CSV and JSON files, apply transformations, and save them as Delta tables in the Bronze layer."*

---

## Objectives

After completing this lab you will be able to:
- Read CSV files with explicit `StructType` schema
- Read JSON files with inferred schema
- Read CSV files with `inferSchema` option
- Apply PySpark transformations: `select`, `withColumn`, `filter`, `concat`, `lower`
- Create temporary views and run SQL aggregation queries
- Save DataFrames as managed Delta tables

---

## Prerequisites

- LAB 01 completed (cluster running, files uploaded to Volume)
- Dataset files accessible via `DATASET_PATH` from `00_setup`

---

## Tasks Overview

Open **`LAB_02_code.ipynb`** and complete the `# TODO` cells.

| Task | What to do | Key concept |
|------|-----------|-------------|
| **Task 1** | Read Customers CSV with Explicit Schema | Define `StructType` with `StructField`, use `.schema()` |
| **Task 2** | Read Orders JSON | `.format("json")` — JSON has self-describing schema |
| **Task 3** | Read Products CSV | `.option("inferSchema", True)` — compare with explicit schema |
| **Task 4** | Transform Customer Data | `.select()`, `.withColumn()`, `concat()`, `lower()`, `.filter()` |
| **Task 5** | Temporary View + SQL Query | `.createOrReplaceTempView()`, `GROUP BY`, `ORDER BY DESC` |
| **Task 6** | Save as Delta Tables (Bronze) | `.write.mode("overwrite").saveAsTable()` |
| **Task 7** | Verify with SQL | `SHOW TABLES IN catalog.schema` |

---

## Detailed Hints

### Task 1: Explicit Schema
- Each `StructField` takes: name (string), type (e.g. `StringType()`), nullable (bool)
- Use `.format("csv")` and `.schema(your_schema)` instead of `.option("inferSchema")`

### Task 2: JSON
- JSON files are self-describing — just use `.format("json")` and `.load(path)`
- No need for schema definition or header option

### Task 3: inferSchema
- The option name is `"inferSchema"` and the value is `True` (or `"true"`)
- Compare the resulting schema types with Task 1 (explicit = all StringType vs inferred types)

### Task 4: Transformations
- `concat(col("first_name"), lit(" "), col("last_name"))` builds full name
- `lower(col("email"))` converts to lowercase
- `.filter(col("email").isNotNull())` removes null emails

### Task 5: TempView + SQL
- Register view: `.createOrReplaceTempView("v_customers")`
- SQL: `SELECT country, COUNT(*) ... GROUP BY country ORDER BY customer_count DESC`

### Task 6: Save as Delta
- Use `.mode("overwrite")` so the lab can be re-run
- Table path format: `{CATALOG}.{BRONZE_SCHEMA}.table_name`

---

## Summary

In this lab you:
- Read CSV (explicit schema) and JSON (inferred schema) files
- Applied transformations: select, withColumn, filter, concat, lower
- Created a temp view and ran SQL aggregation queries
- Saved 3 Delta tables in the Bronze layer

> **Exam Tip:** `inferSchema` reads the file twice (once for schema, once for data). Always prefer explicit schema in production. JSON and Parquet have embedded schemas.

> **What's next:** In LAB 03 you will use MERGE, UPDATE, DELETE on Delta tables and explore Time Travel for disaster recovery.

# LAB 02: ELT Ingestion & Transformations

**Duration:** ~40 min  
**Day:** 1  
**After module:** M02: ELT Data Ingestion  
**Difficulty:** Beginner-Intermediate

---

## Scenario

> *"The RetailHub data team received the first data export from the store system -- CSV files with customer and product data, plus a JSON file with order history. Your task: load these files into the Lakehouse, apply basic transformations, and save them as Delta tables in the Bronze layer."*

---

## Objectives

After completing this lab you will be able to:
- Read CSV, JSON, and Parquet files using `spark.read`
- Compare `inferSchema` vs explicit schema definition
- Apply DataFrame transformations: select, withColumn, filter, cast
- Create temporary views and run SQL queries
- Write DataFrames as managed Delta tables using `saveAsTable()`

---

## Prerequisites

- Completed LAB 01 (or just run `%run ../setup/00_setup`)
- Dataset files available in Volume or `DATASET_PATH`

---

## Part 1: Data Ingestion (~15 min)

### Task 1: Read Customers CSV

Read the customers CSV file with explicit schema (don't use `inferSchema`).

**Requirements:**
- Define a `StructType` schema with appropriate types
- Use `.option("header", True)`
- Verify column types with `printSchema()`

> **Exam Tip:** `inferSchema` triggers an extra pass over the data. In production, always define schema explicitly for CSV files.


### Task 2: Read Orders JSON

Read the orders batch JSON file. JSON infers schema automatically, but verify nested structures.

**Requirements:**
- Use `spark.read.format("json")`
- Check if any columns contain nested types (struct, array)

### Task 3: Read Products CSV

Read the products file with `inferSchema` and then compare the schema with a manually defined one.

---

## Part 2: Transformations (~15 min)

### Task 4: Clean Customer Data

Apply the following transformations to `df_customers`:
1. `select` only: customer_id, first_name, last_name, email, city, country
2. `withColumn` -- create `full_name` by concatenating first_name + " " + last_name
3. `withColumn` -- convert email to lowercase using `lower()`
4. `filter` -- keep only rows with non-null email

### Task 5: Create Temp View + SQL Query

1. Register `df_customers` as a temporary view `v_customers`
2. Write a SQL query to count customers per country
3. Display the result sorted by count descending

> **Exam Tip:** `createOrReplaceTempView()` is session-scoped. `createOrReplaceGlobalTempView()` is accessible from any notebook on the same cluster via `global_temp.view_name`.

---

## Part 3: Save as Delta Tables (~10 min)

### Task 6: Write to Bronze Layer

Save all three DataFrames as Delta tables in the Bronze schema:
- `bronze.customers`
- `bronze.orders`
- `bronze.products`

**Requirements:**
- Use `.write.mode("overwrite").saveAsTable()`
- Use fully qualified names: `{CATALOG}.{BRONZE_SCHEMA}.table_name`

### Task 7: Verify Tables

Query the tables using SQL to verify they were saved correctly.

---

## Summary

In this lab you:
- Loaded CSV and JSON files into Spark DataFrames
- Applied transformations: select, withColumn, filter, cast
- Created temporary views and ran SQL queries
- Saved cleaned data as Delta tables in the Bronze layer

> **What's next:** In LAB 03 you will use MERGE to handle incremental data updates and learn Time Travel for disaster recovery.